# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
print(os.getenv('HF_ENDPOINT'))
os.environ['HF_DATASETS_CACHE'] = '/mnt/windows/Linux/huggingface/datasets/'
print(os.getenv('HF_DATASETS_CACHE'))
os.environ['HF_HOME'] = '/mnt/windows/Linux/huggingface/'
print(os.getenv('HF_HOME'))
# os.environ['HF_HUB_OFFLINE']='0'

https://hf-mirror.com
/mnt/windows/Linux/huggingface/datasets/
/mnt/windows/Linux/huggingface/


In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/home/chenglim/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/chenglim/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/home/chenglim/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是八卦之一，也是八宫图之一，其符号是“天”，代表阳、强、刚、动、创造、发展等。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雷、雨等自然现象，也象征着父亲、领导、力量、刚强等。乾卦的意义包括：

1. 创造与领导：乾卦象征着创造和领导的力量，代表着阳刚之气。在周易中，乾卦主宰着八宫，象征着君主、领导、父亲等权力人物。
2. 坚定与进展：乾卦也象征着坚定和进展。因为它的组合是“天”，天是代表稳定和坚定的象征。乾卦的能量可以使人或事物更加坚定，有助于进展和成功。
3. 变化与适应：乾卦虽然以阳刚为主，但它也包含着变化和适应的能力。因为乾卦是由两个阴爻夹一个阳爻构成，这使得它可以适应不同的环境和情况，具有变化的能力。

总之，乾卦是周易中最受欢迎和最具有代表性的八卦之一，代表着创造、领导、坚定、进展、变化等。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [9]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它的卦象是由两个阴爻夹一个阳爻构成，象征着天与地之间的争斗。因此，讼卦代表了诉讼、争端、矛盾、争议等纷争现象。同时，讼卦也象征着通过协商、沟通、妥协等手段解决纷争的能力。

具体来说，讼卦的意义包括：

1. 争端与诉讼：讼卦象征着争端和诉讼。在周易中，讼卦主宰着八宫，象征着法律、司法、诉讼等。因此，讼卦也代表了通过诉讼解决纷争的能力。
2. 沟通与妥协：讼卦也象征着沟通和妥协。因为它的组合是“天”，天是代表沟通和妥协的象征。讼卦的能量可以使人或事物更加开放、包容和妥协，有助于解决纷争。
3. 判断与正义：讼卦还象征着判断和正义。在周易中，讼卦的卦象是由两个阴爻夹一个阳爻构成，象征着天地之间的大小、强弱、善恶等。因此，讼卦也代表了通过判断和正义来解决纷争的能力。

总之，讼卦是周易中最具有争议和挑战的八卦之一，代表了争端、诉讼、矛盾、争议等。但它也具有解决纷争的能力，通过沟通、妥协和判断来达到和平解决的能力。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [11]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250301_004056"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [12]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [13]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由六个阳爻组成，象征着天。它代表着刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250301_004056）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [14]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250301_004056）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [15]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
挂一卦，是中国周易中的第一卦，由乾卦（天）和坤卦（地）组合而成，象征着天地相交，代表着一个普遍的哲理，即天地相互依存，相互感应。挂一卦的核心哲学是：天地之间的交互作用，是自然界的普遍现象，而人类应当效仿天地交互的作用，以实现人类社会的和谐。挂一卦的核心卦象为下卦（地）上卦（天），代表着天地相交，相互感应的景象。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250301_004056）：
[gMASK]sop 师卦是什么？ 师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。
